In [1]:
! pip install chardet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.4 MB/s eta 0:00:00


In [2]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional
from keras.models import Sequential

In [3]:
import chardet

# Определение кодировки
with open("/content/Tolstoy Lev. Metel - BooksCafe.Net.txt", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
    print(result)

# Использование определенной кодировки для чтения файла
with open("/content/Tolstoy Lev. Metel - BooksCafe.Net.txt", 'r', encoding=result['encoding']) as _in:
    lines = []
    for line in _in:
        line = line.strip().lower()  # Приводим к нижнему регистру
        if len(line) == 0:  # Пропускаем пустые строки
            continue
        lines.append(line)

text = " ".join(lines)

# Создаем множество уникальных символов в тексте (включая кириллицу)
chars = set(text)
nb_chars = len(chars)

print(f"Количество уникальных символов: {nb_chars}")
print(f"Список уникальных символов: {chars}")


{'encoding': 'windows-1251', 'confidence': 0.99, 'language': 'Russian'}
Количество уникальных символов: 59
Список уникальных символов: {'о', '4', 'с', '3', ' ', 'е', 'у', 'р', 'ю', '2', 'ъ', 'т', 'к', ')', 'ж', 'ы', 'й', 'в', ';', '.', '&', 'г', '_', 'х', '5', 'ч', '?', '«', 'ш', 'м', ',', '–', 'з', '\xa0', 'а', 'u', 'я', 'б', '-', '!', 'v', ':', 'д', 'и', 'щ', 'л', 'x', '…', '#', 'н', 'i', 'п', 'ц', '(', 'ф', 'ь', 'э', 'ё', '»'}


In [4]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [5]:
# для удобства выберете фиксированную длину последовательность 10 символов
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [6]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 15 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


In [7]:
%%time

# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]


    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
421/421 [==============================] - 7s 12ms/step - loss: 2.9050
Генерация из посева: ёпло, да м
ёпло, да ме сто поло поло поло поло поло поло поло поло поло поло поло поло поло поло поло поло поло поло поло==================================================
Итерация #: 1
421/421 [==============================] - 5s 12ms/step - loss: 2.4880
Генерация из посева: едет, что 
едет, что предери породо предери породо предери породо предери породо предери породо предери породо предери по==================================================
Итерация #: 2
421/421 [==============================] - 5s 12ms/step - loss: 2.3667
Генерация из посева: ь, средь г
ь, средь го подораза сталь о породол са подораза сталь о породол са подораза сталь о породол са подораза сталь==================================================
Итерация #: 3
421/421 [==============================] - 5s 12ms/step - loss: 2.2883
Генерация из посева: еня, то ор
еня, то ородол соворивали но быловали но быловали

In [8]:
import tensorflow as tf
model2 = Sequential()
model2.add(
    Bidirectional(GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    ))
)
model2.add(Dense(nb_chars))
model2.add(Activation("softmax"))
model2.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [9]:
X = tf.cast(X, dtype=tf.float32)
y = tf.cast(y, dtype=tf.float32)

In [10]:
%%time
# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model2.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
421/421 [==============================] - 10s 13ms/step - loss: 2.9020
Генерация из посева: льзя броса
льзя бросать в советчик вородно было ветер и на стороны и не вотрим не вотрим не вотрим не вотрим не вотрим не==================================================
Итерация #: 1
421/421 [==============================] - 6s 13ms/step - loss: 2.4747
Генерация из посева: да; и трет
да; и третки, и подноги и подноги и подноги и подноги и подноги и подноги и подноги и подноги и подноги и подн==================================================
Итерация #: 2
421/421 [==============================] - 6s 13ms/step - loss: 2.3598
Генерация из посева: ерез светл
ерез светло в семет не вот и подноги и подноги и подноги и подноги и подноги и подноги и подноги и подноги и п==================================================
Итерация #: 3
421/421 [==============================] - 6s 13ms/step - loss: 2.2824
Генерация из посева: ь на спину
ь на спину и на стороны и не вотрим не вотрим не

### Непосредсвенно изменения модели для улучшения качества генерации текса

In [28]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam


In [34]:
# Пример данных
input_chars = list(text)
nb_chars = len(set(chars))

char2index = {ch: i for i, ch in enumerate(sorted(set(chars)))}
index2char = {i: ch for ch, i in char2index.items()}

In [35]:
import re

# функция для выбора посева, начиная с границы слова
def get_random_seed(input_text, seed_length):
    words = re.findall(r'\b\w+\b', input_text)  # находим все слова в тексте
    random_word = np.random.choice(words)  # выбираем случайное слово
    start_idx = input_text.find(random_word)  # находим индекс начала этого слова

    # проверяем, чтобы выборка включала это слово и далее вплоть до длины посева
    return input_text[start_idx:start_idx + seed_length]

In [36]:
# Функция для сэмплирования с температурой
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [37]:
SEQLEN = 20  # Устанавливаем длину последовательности в 20 символов
HIDDEN_UNITS = 512
TEMPERATURE = 0.5
NUM_ITERATIONS = 10
BATCH_SIZE = 128
NUM_PREDS_PER_EPOCH = 100

In [38]:
# Генерация данных X и y с учетом SEQLEN = 20
X = np.zeros((len(input_chars) - SEQLEN, SEQLEN, nb_chars), dtype=np.bool_)
y = np.zeros((len(input_chars) - SEQLEN, nb_chars), dtype=np.bool_)

for i in range(len(input_chars) - SEQLEN):
    for j in range(SEQLEN):
        X[i, j, char2index[input_chars[i + j]]] = 1
    y[i, char2index[input_chars[i + SEQLEN]]] = 1

In [39]:
# Создаем модель
model = Sequential()
model.add(LSTM(HIDDEN_UNITS, return_sequences=True, input_shape=(SEQLEN, nb_chars)))
model.add(Dropout(0.2))
model.add(LSTM(HIDDEN_UNITS))
model.add(Dense(nb_chars, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

In [41]:
# Тренировка и генерация текста
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=1)

    test_chars = get_random_seed(text, SEQLEN)
    generated_text = test_chars
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")

    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[sample(pred, TEMPERATURE)]

        generated_text += y_pred
        test_chars = test_chars[1:] + y_pred
        print(y_pred, end="")

    print("\nСгенерированный текст: %s" % generated_text)
    print()

Итерация #: 0
421/421 [==============================] - 94s 223ms/step - loss: 2.3719
Генерация из посева: заметна стала только
заметна стала только сталсе стариваль м пробол прастой тоже тосеть в терыл да ка скакал сленае на презими сане и вает по
Сгенерированный текст: заметна стала только сталсе стариваль м пробол прастой тоже тосеть в терыл да ка скакал сленае на презими сане и вает по

Итерация #: 1
421/421 [==============================] - 94s 224ms/step - loss: 2.1217
Генерация из посева: его дороги не видать
его дороги не видать с водем и неговорения его восто но одно как долокольчика мо одолонитывали подкулая снига станивал м
Сгенерированный текст: его дороги не видать с водем и неговорения его восто но одно как долокольчика мо одолонитывали подкулая снига станивал м

Итерация #: 2
421/421 [==============================] - 94s 222ms/step - loss: 1.9153
Генерация из посева: посинелым лицом, жид
посинелым лицом, жидовить, как меня и все станную, подне голос были подпрогивала 

### Вывод
Так как английским я владею, на сколько мне позволяет яндекс перводчик, в качестве обрабатываемого текса я использовал рассказ Л.Н. Толстого "Метель".

Глядя на сгенерированный текст нашей модели, и сравнив его с первым вариантом, можно наблюдать, явное улучшение связанности текса. Некоторые слова, коненчно, все еще кажутся непонятными, но это можно связать с форматом текста и содержанием слов, не всегда понятных для современного человека :)))

Что было предпринято в модернизации модели и кода в целом, или что могло бы помочь сдлеать модель лучше:
1. **Увеличение объема данных** . Чем больше данных, тем лучше модель сможет понять контексты и закономерности в языке.
2. **Увеличение длины последовательности** (SEQLEN)
Увеличили длину последовательности, которую модель использует для предсказания следующего символа или слова. Это позволило модели учитывать больше контекста при предсказаниях.
3. **Увеличение размерности скрытых слоев**
Увеличили количество нейронов в скрытых слоях модели (например, LSTM- или GRU-слои). Это позволило модели лучше захватывать сложные зависимости в данных.
4. **Использование методов регуляризации**
  *Dropout* и *L2-регуляризация*: Добавление регуляризации может помочь модели избежать переобучения, что улучшит генерализацию и связность текста.
5. **Увеличение числа эпох** (NUM_EPOCHS_PER_ITERATION)
Увеличили количество эпох обучения в каждой итерации. Это дало модели больше времени для обучения на данных и отняло его у меня :))).
6. **Техника температурного сэмплирования**
 Температурное сэмплирование позволяет контролировать степень «креативности» предсказаний. Более низкая температура делает предсказания модели более консервативными и предсказуемыми, в то время как высокая температура добавляет разнообразия.
7. **Оптимизация гиперпараметров**
Изменили параметры обучения: скорость обучения (learning rate) и размер батча (batch size).

И, наверное, самый радикальный способ - использовать предобученные модели (например, GPT или BERT) и дообучить их на вашей задаче.